<a id='h22'></a>
## Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<h1>Содержание (кликабельное):</h1>

<a href='#h22'>1. Название проекта</a><br>
<a href='#h2'>2. Загрузка и подготовка данных</a><br>
&nbsp;&nbsp;&nbsp;<a href='#h31'>2.1 Вывод по шагу 2</a><br>
<a href='#h3'>3. Обучение и проверка модели</a><br>
&nbsp;&nbsp;&nbsp;<a href='#h32'>3.1 Вывод по шагу 3</a><br>
<a href='#h41'>4. Подготовка к расчёту прибыли</a><br>
&nbsp;&nbsp;&nbsp;<a href='#h4'>4.1 Вывод по шагу 4</a><br>
<a href='#h5'>5. Расчёт прибыли и рисков</a><br>
&nbsp;&nbsp;&nbsp;<a href='#h51'>5.1 Вывод по шагу 5</a><br>
<a href='#h6'>6. Общий вывод</a><br>

<a id='h2'></a>
## Загрузка и подготовка данных

In [1]:
!pip install scikit-learn==1.1.3

     |████████████████████████████████| 30.8 MB 50 kB/s  eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    f1_score, 
    mean_squared_error, 
    roc_auc_score,
    precision_score,
    recall_score,
    roc_curve
)

import warnings
warnings.filterwarnings("ignore")

state = 55555

Каждый регион загрузим в отдельный датафрейм

In [3]:
geo_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_0.describe(include='all')

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99990,NaN,NaN,NaN,NaN
top,bxg6G,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,0.500419,0.250143,2.502647,92.500000
std,NaN,0.871832,0.504433,3.248248,44.288691
min,NaN,-1.408605,-0.848218,-12.088328,0.000000
25%,NaN,-0.072580,-0.200881,0.287748,56.497507
50%,NaN,0.502360,0.250252,2.515969,91.849972
75%,NaN,1.073581,0.700646,4.715088,128.564089


In [4]:
geo_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [5]:
geo_0.duplicated().sum()

0

In [6]:
geo_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_1.describe(include='all')

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99996,NaN,NaN,NaN,NaN
top,wt4Uk,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,1.141296,-4.796579,2.494541,68.825000
std,NaN,8.965932,5.119872,1.703572,45.944423
min,NaN,-31.609576,-26.358598,-0.018144,0.000000
25%,NaN,-6.298551,-8.267985,1.000021,26.953261
50%,NaN,1.153055,-4.813172,2.011479,57.085625
75%,NaN,8.621015,-1.332816,3.999904,107.813044


In [7]:
geo_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [8]:
geo_1.duplicated().sum()

0

In [9]:
geo_2 = pd.read_csv('/datasets/geo_data_2.csv')
geo_2.describe(include='all')

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99996,NaN,NaN,NaN,NaN
top,xCHr8,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,0.002023,-0.002081,2.495128,95.000000
std,NaN,1.732045,1.730417,3.473445,44.749921
min,NaN,-8.760004,-7.084020,-11.970335,0.000000
25%,NaN,-1.162288,-1.174820,0.130359,59.450441
50%,NaN,0.009424,-0.009482,2.484236,94.925613
75%,NaN,1.158535,1.163678,4.858794,130.595027


In [10]:
geo_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [11]:
geo_2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


Посмотрим на столбец ID

In [12]:
geo_0['id'].value_counts()

bxg6G    2
AGS9W    2
74z30    2
fiKDv    2
A5aEY    2
        ..
s9RxW    1
RvZkL    1
NmFE7    1
9pMTP    1
LirZ2    1
Name: id, Length: 99990, dtype: int64

In [13]:
geo_1['id'].value_counts()

wt4Uk    2
5ltQ6    2
LHZR0    2
bfPNe    2
ztitl    1
        ..
DrI5o    1
nCUT7    1
b2KEk    1
WbnfY    1
ytln6    1
Name: id, Length: 99996, dtype: int64

In [14]:
geo_2['id'].value_counts()

xCHr8    2
VF7Jo    2
KUPhW    2
Vcm5J    2
FJoxU    1
        ..
KgPW0    1
LC7Ey    1
Ue0E9    1
0u8C0    1
XcamQ    1
Name: id, Length: 99996, dtype: int64

In [15]:
geo_2.duplicated().sum()

0

In [16]:
geo_2.loc[geo_2['id'] == 'xCHr8']

,id,f0,f1,f2,product
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641


В данных отсутсвуют пропуски и дубликаты. Удалим столбец id. Он не несет значимости для модели.

In [17]:
geo_0 = geo_0.drop(['id'], axis=1)
geo_1 = geo_1.drop(['id'], axis=1)
geo_2 = geo_2.drop(['id'], axis=1)

Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе. 

Наверное можно сразу подсчать среднее по региону и удалить те ячейчеки,где product == 0.

Наши модели можно было бы обучить быстрее, но как это скажется на проекте я не знаю, поэтоу оставлю без изменения.

Вопрос почему в product == 0, тут или не указали данные или там просто нету сырья.

In [18]:
## средний запас продукта по регионам.
geo_0_mean = geo_0['product'].mean()
print('Средний запас продукта в 1 регионе', geo_0_mean)
geo_1_mean = geo_1['product'].mean()
print('Средний запас продукта в 2 регионе', geo_1_mean)
geo_2_mean = geo_2['product'].mean()
print('Средний запас продукта в 3 регионе', geo_2_mean)


Средний запас продукта в 1 регионе 92.50000000000001
Средний запас продукта в 2 регионе 68.82500000000002
Средний запас продукта в 3 регионе 95.00000000000004


In [19]:
## Разобьем на выборки 
## первый регион
target_g0 = geo_0['product']
features_g0 = geo_0.drop('product', axis=1)

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_g0, target_g0, test_size=0.25,  random_state=state) 

## второй регион
target_g1 = geo_1['product']
features_g1 = geo_1.drop('product', axis=1)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_g1, target_g1, test_size=0.25,  random_state=state) 
## Третий регион
target_g2 = geo_2['product']
features_g2 = geo_2.drop('product', axis=1)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_g2, target_g2, test_size=0.25,  random_state=state) 


In [20]:
print(features_g0.shape)
print(features_g1.shape)
print(features_g2.shape)

(100000, 3)
(100000, 3)
(100000, 3)


In [21]:
features_g2.columns

Index(['f0', 'f1', 'f2'], dtype='object')

Приведем все признаки к одному масштабу.

In [22]:
def f_scaler(train, valid):
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(train[numeric])
    train[numeric] = scaler.transform(train[numeric])
    valid[numeric] = scaler.transform(valid[numeric])
    return train, valid

In [23]:
#первый регион
features_train_0,features_valid_0 = f_scaler(features_train_0,features_valid_0)

In [24]:
#второй регион
features_train_1,features_valid_1 = f_scaler(features_train_1,features_valid_1)

In [25]:
#третий регион
features_train_2,features_valid_2 = f_scaler(features_train_2,features_valid_2)

<a id='h31'></a>
### Вывод по шагу 1.<br>
Данные готовы к обучению. Определены : Признаки для модели:f0,f1,f2.Целевой признак: product.<br>
Удален фиктивный признак 'id'. Данные проверены на пропуски и дубликаты, которых в наших данных нету. 


<a id='h3'></a>
## Обучение и проверка модели

In [26]:
#обучим модели

In [27]:
#первый регион
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predicted_target_0 = model_0.predict(features_valid_0)

geo_pm0 = geo_0['product'].mean()
geo_pred_target_m0 = predicted_target_0.mean()
RMSE_0 = mean_squared_error(target_valid_0, predicted_target_0)**0.5
print('Средний текущий запас сырья', geo_pm0)
print('Средний предсказанный запас сырья', geo_pred_target_m0)
print('RMSE-0 ', RMSE_0)

Средний текущий запас сырья 92.50000000000001
Средний предсказанный запас сырья 92.29980563364998
RMSE-0  37.572787216783894


In [28]:
#Второй регион
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predicted_target_1 = model_1.predict(features_valid_1)

geo_pm1 = geo_1['product'].mean()
geo_pred_target_m1 = predicted_target_1.mean()
RMSE_1 = mean_squared_error(target_valid_1, predicted_target_1)**0.5
print('Средний текущий запас сырья', geo_pm1)
print('Средний предсказанный запас сырья', geo_pred_target_m1)
print('RMSE-1 ', RMSE_1)


Средний текущий запас сырья 68.82500000000002
Средний предсказанный запас сырья 68.65786052898808
RMSE-1  0.8926018653538472


In [29]:
#Третий регион
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predicted_target_2 = model_2.predict(features_valid_2)

geo_pm2 = geo_2['product'].mean()
geo_pred_target_m2 = predicted_target_2.mean()
RMSE_2 = mean_squared_error(target_valid_2, predicted_target_2)**0.5
print('Средний текущий запас сырья', geo_pm2)
print('Средний предсказанный запас сырья', geo_pred_target_m2)
print('RMSE-2 ', RMSE_2)

Средний текущий запас сырья 95.00000000000004
Средний предсказанный запас сырья 95.04059153913464
RMSE-2  40.02536281319398


<a id='h32'></a>
### Вывод по шагу 3: 


Лучший параметр RMSE показывает модель во 2 регионе, а наибольший средний запас сырья в 3 регионе.

<a id='h4'></a>
## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [30]:
extraction_points = 500
extraction_best_point = 200
budget = 10000000000 / 1000000
income_1_bar = 450000 / 1000000
product_income = 450000 #for 1000 bar 
loss_probability = 0.025

In [31]:
#Объём сырья для безубыточной разработки новой скважины:
break_even_point = budget / extraction_best_point / product_income
print('Точка безубыточности', break_even_point)

Точка безубыточности 0.00011111111111111112


In [32]:
print('Средний объем запасов в 0-м регионе: ',geo_0['product'].mean())
print('Средний объем запасов в 1-м регионе: ', geo_1['product'].mean())
print('Средний объем запасов в 2-м регионе: ', geo_2['product'].mean())

Средний объем запасов в 0-м регионе:  92.50000000000001
Средний объем запасов в 1-м регионе:  68.82500000000002
Средний объем запасов в 2-м регионе:  95.00000000000004


<a id='h41'></a>
### Вывод по шагу 4:

Средний обьем запасов в каждом из регионов в 1 регионе - 92.5, во втором - 68.82, в третьем регионе - 95.0, ниже точки безубыточности равной 111.111.

<a id='h5'></a>
## Расчёт прибыли и рисков 

In [33]:
# создаём функцию для подсчета прибыли
def profit(target, pred, count, budget):
    bore_sort = pred.sort_values(ascending=False)
    picked= target[bore_sort.index][:count] 
    reven = income_1_bar * picked.sum() 
    profit = reven - budget 
    return profit

In [34]:
def check_region(target, predictions):
    predictions = pd.Series(predictions)
    values = []
    state = np.random.RandomState(12345)
    for i in range(1000):
        target_subsample = target.reset_index(drop=True).sample(n=extraction_points, replace=True, random_state=state)
        pred_subsample = predictions[target_subsample.index]
        value = profit(target_subsample, pred_subsample, extraction_best_point, budget)
        values.append(value)

    # считаем количество убытков
    values = pd.Series(values)
    risk = (values < 0).mean()

    # считаем доверительный интервал
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)

    # выводим результат
    print('Средняя прибыль в регионе:', round(values.mean(), 2), 'млн. руб')
    print(f'95% доверительный интервал: ({lower}, {upper})')
    print('Риск убытков: {:.1%}'.format((values < 0).mean()))

    # сравниваем с необходимым риском
    if (risk * 100) <= 2.5:
          print('Риск убытков меньше необходимого значения. Регион можно выбрать для разработки.')
    else:
          print('Риск убытков больше необходимого значения.')      

In [35]:
check_region(target_valid_0, predicted_target_0)


Средняя прибыль в регионе: 514.26 млн. руб
95% доверительный интервал: (-16.041709134708928, 1055.7298739531043)
Риск убытков: 3.2%
Риск убытков больше необходимого значения.


In [36]:
raspred_1 = check_region(target_valid_1, predicted_target_1)
raspred_1

Средняя прибыль в регионе: 489.94 млн. руб
95% доверительный интервал: (82.35348420987361, 903.6774010184937)
Риск убытков: 0.6%
Риск убытков меньше необходимого значения. Регион можно выбрать для разработки.


In [37]:
raspred_2 = check_region(target_valid_2, predicted_target_2)

Средняя прибыль в регионе: 416.02 млн. руб
95% доверительный интервал: (-159.79631679365286, 978.7860422714596)
Риск убытков: 8.7%
Риск убытков больше необходимого значения.


<a id='h51'></a>
### Вывод по шагу 5: 


Из первого региона можно получать наибольшую прибыль, но Риск убытков: 3.2%, что не соответствует условию задачи меньше 2.5%.
Хоть второй регион и является наименьшим по количеству запасов, но и Среднеквадратичная ошибка RMSE в этом регионе наименьшая 0.89, вычисленный риск убытков в этом регионе 0.6%. 
И именно этот регион и предлагается для разработки на основании проведенного исследования

<a id='h6'></a>
## Общий вывод.

В данном проекте мы изучили содержимое таблиц. Убедились в отсутствии пропусков и дубликатов, избавились от фиктивных признаков. <br>
Разделили данные каждого региона на обучающую и валидационную выборки (в соотношении 75:25), а также на признаки и целевой признак. Привели признаки к единому масштабу для данных каждого региона. <br>
Рассчитали Средний предсказанный запас сырья и Среднеквадратичная ошибку для каждого региона. <br>
И также точку безубыточности<br>
Расчёт прибыль и риски для каждого региона. <br>
Определили, что второй регион можно выбрать для разработки так как он соответствует условию по рискам меньше 2.5%

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- []  Весь код выполняется без ошибок
- []  Ячейки с кодом расположены в порядке исполнения
- []  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован